# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [13]:
import numpy as np
from numpy import mean
import pandas as pd
import nltk
import pickle
import os
from keras.models import Sequential

In [6]:
os.getcwd()

'C:\\Users\\ooles\\OneDrive\\Desktop'

In [16]:
os.chdir('/Users/ooles/OneDrive/Desktop/')

In [14]:
df = pd.read_pickle("df_clean_newwer_new.pkl")
df.shape

(333459, 10)

In [15]:
df.head()

,identifier,speaker_id,file,corpus,configuration,machine,reference.text,hypothesis.text,scoring.wer,recomputed_wer
1,sw2061A-ms98-a-0123,1167,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,yeah because see what happens is they have a g...,yeah because see what happens is they have a g...,0.000000,0.000000
2720,sw2831A-ms98-a-0046,1139,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,i think people are going to follow the stock m...,i think people are going to follow the stock m...,0.000000,0.000000
2721,sw2567B-ms98-a-0068,1159,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,good don't ever drink scotch it's terrible,good don't ever drink scott it's terrible,0.142857,0.142857
2723,sw4276B-ms98-a-0033,1532,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,so they're going to continue,so they're going to continue,0.000000,0.000000
2727,sw3117B-ms98-a-0031,1312,amazon__8000_8__switchboard_segmented.json,switchboard_segmented,amazon__8000_8,amazon,and see back when i went to school there when ...,and see back when i went to school there when ...,0.133333,0.133333


In [5]:
%load_ext autoreload
%aimport helper, tests
%autoreload 1

In [16]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 447858824528633175
]


## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [93]:
# Load English data
english_sentences = helper.load_data('eng.txt')
# Load French data
french_sentences = helper.load_data('fre.txt')

print('Dataset Loaded')

Dataset Loaded


In [68]:
df["reference.text"].iloc[3]

"so they're going to continue"

In [79]:
ref=df["reference.text"].values.tolist()

In [57]:
#r=str(ref).strip('[]')

In [61]:
#r=r.replace('\'', ' ')

In [62]:
#r=r.replace(',', '.')

In [64]:
#r=r.replace('"', ' ')

In [92]:
#df["reference.text"] = df["reference.text"].apply(lambda x: x.replace(',',"."))

In [90]:
df["reference.text"].to_csv(r'/Users/ooles/OneDrive/Desktop/fre.txt', header=None, index=None, sep='.', mode='a')

C:\Users\ooles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [91]:
df["hypothesis.text"].to_csv(r'/Users/ooles/OneDrive/Desktop/eng.txt', header=None, index=None, sep='.', mode='a')

C:\Users\ooles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [80]:
hyp=df["hypothesis.text"].values.tolist()

In [74]:
#h=str(hyp).strip('[]')
#h=h.replace('\'', ' ')
#h=h.replace('"', ' ')
#h=h.replace(',', '.')


In [29]:
tokenized_sents_r = [nltk.word_tokenize(i) for i in ref]

In [30]:
tokenized_sents_h = [nltk.word_tokenize(i) for i in hyp]

In [25]:
tokenized_sent_ref=[x for s in tokenized_sents_r for x in s] 

In [31]:
tokenized_sent_hyp=[x for s in tokenized_sents_h for x in s] 

In [27]:
#tokenized_sent_ref

### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [96]:
for sample_i in range(6):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  yeah because see what happens is they have a good backup policy and anybody can fix it
small_vocab_fr Line 1:  yeah because see what happens is they have a good backup policy and anybody can fix it
small_vocab_en Line 2:  i think people are going to follow the stock market if if it takes off
small_vocab_fr Line 2:  i think people are going to follow the stock market if if it takes off
small_vocab_en Line 3:  good don't ever drink scott it's terrible
small_vocab_fr Line 3:  good don't ever drink scotch it's terrible
small_vocab_en Line 4:  so they're going to continue
small_vocab_fr Line 4:  so they're going to continue
small_vocab_en Line 5:  and see back when i went to school there when i first entered the school it cost me $200 has an out as an out of state student
small_vocab_fr Line 5:  and see back when i went to school there when i first entered the school it cost me two hundred dollars as an out as an out of state student
small_vocab_en Line 6:  which wou

From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [97]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

4636241 English words.
49165 unique English words.
10 Most common words in the English dataset:
"the" "and" "to" "of" "a" "i" "that" "you" "in" "it"

4857434 French words.
19255 unique French words.
10 Most common words in the French dataset:
"the" "and" "to" "of" "a" "i" "that" "you" "it" "in"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [98]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer(char_level = False)
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk



In [100]:
# Tokenize Example output
#text_sentences = [
   # 'The quick brown fox jumps over the lazy dog .',
   # 'By Jove , my quick study of lexicography won a prize .',
   # 'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(english_sentences)
print(text_tokenizer.word_index)
#print()
#for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
   # print('Sequence {} in x'.format(sample_i + 1))
   # print('  Input:  {}'.format(sent))
   # print('  Output: {}'.format(token_sent))

{'the': 1, 'and': 2, 'to': 3, 'a': 4, 'of': 5, 'i': 6, 'that': 7, 'you': 8, 'in': 9, 'it': 10, 'is': 11, 'was': 12, 'we': 13, 'for': 14, 'so': 15, 'but': 16, 'he': 17, 'have': 18, 'this': 19, 'with': 20, 'on': 21, 'be': 22, 'they': 23, 'not': 24, 'as': 25, 'what': 26, "it's": 27, 'know': 28, 'at': 29, 'are': 30, 'one': 31, 'like': 32, 'his': 33, 'do': 34, 'if': 35, 'all': 36, 'my': 37, 'had': 38, 'can': 39, 'or': 40, 'just': 41, 'there': 42, 'think': 43, 'yeah': 44, 'me': 45, 'she': 46, 'then': 47, 'about': 48, 'from': 49, 'would': 50, 'no': 51, 'by': 52, 'out': 53, 'him': 54, 'them': 55, 'when': 56, "don't": 57, 'her': 58, 'an': 59, "that's": 60, 'which': 61, 'up': 62, 'well': 63, 'will': 64, 'more': 65, 'your': 66, 'were': 67, 'now': 68, 'people': 69, 'said': 70, 'some': 71, 'get': 72, "i'm": 73, 'because': 74, 'time': 75, 'how': 76, 'going': 77, 'who': 78, 'good': 79, 'could': 80, 'very': 81, 'has': 82, 'go': 83, 'see': 84, 'their': 85, 'been': 86, 'really': 87, 'into': 88, 'somethi

In [103]:
#print(text_tokenized)

### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [104]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen = length, padding = 'post')



In [84]:
from tests import *

In [105]:
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
#for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
 #   print('Sequence {} in x'.format(sample_i + 1))
 #   print('  Input:  {}'.format(np.array(token_sent)))
 #   print('  Output: {}'.format(pad_sent))

### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [106]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk



In [107]:
preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

In [109]:
print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 173
Max French sentence length: 170
English vocabulary size: 47415
French vocabulary size: 18948


## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an optional Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [110]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [111]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3
    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences = True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)
    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model


In [122]:
# Reshaping the input to work with a basic RNN
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [123]:
tmp_x.shape

(333460, 170, 1)

In [118]:
max_french_sequence_length

170

In [119]:
english_vocab_size

47415

In [120]:
french_vocab_size

18948

In [ ]:
# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=10, validation_split=0.2)




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 266768 samples, validate on 66692 samples
Epoch 1/10







In [ ]:
# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [17]:
from keras.models import Sequential
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    rnn = GRU(64, return_sequences=True, activation="tanh")
    
    embedding = Embedding(french_vocab_size, 64, input_length=input_shape[1]) 
    logits = TimeDistributed(Dense(french_vocab_size, activation="softmax"))
    
    model = Sequential()
    #em can only be used in first layer --> Keras Documentation
    model.add(embedding)
    model.add(rnn)
    model.add(logits)
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    
    return model


In [18]:
tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(preproc_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))



In [19]:
tmp_x.shape

(137861, 21)

In [20]:
max_french_sequence_length

21

In [21]:
english_vocab_size

199

In [22]:
french_vocab_size

345

In [26]:
# TODO: Train the neural network

embeded_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)

embeded_model.fit(tmp_x, preproc_french_sentences, batch_size=32, epochs=10, validation_split=0.2)



Train on 110288 samples, validate on 27573 samples
Epoch 1/10
  7872/110288 [=>............................] - ETA: 3:41 - loss: 3.2057 - acc: 0.4269

KeyboardInterrupt: 

In [ ]:

# TODO: Print prediction(s)
print(logits_to_text(embeded_model.predict(tmp_x[:1])[0], french_tokenizer))

### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [9]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(Bidirectional(GRU(128, return_sequences = True, dropout = 0.1), 
                           input_shape = input_shape[1:]))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
#tests.test_bd_model(bd_model)




In [10]:
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])

In [12]:
tmp_x[0]

array([  26,  789,   11,   23,   18,    4,   79, 5363, 1795,    2, 1236,
         39, 2085,   10,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

In [15]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [11]:
# TODO: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences, preproc_french_sentences.shape[1])
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2], 1))

In [12]:


bidi_model = bd_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)









Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.




In [ ]:

bidi_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 266768 samples, validate on 66692 samples
Epoch 1/20







In [ ]:
# Print prediction(s)
print(logits_to_text(bidi_model.predict(tmp_x[:1])[0], french_tokenizer))

### Model 4: Encoder-Decoder (OPTIONAL)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [11]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # OPTIONAL: Implement
    learning_rate = 1e-3
    model = Sequential()
    model.add(GRU(128, input_shape = input_shape[1:], return_sequences = False))
    model.add(RepeatVector(output_sequence_length))
    model.add(GRU(128, return_sequences = True))
    model.add(TimeDistributed(Dense(french_vocab_size, activation = 'softmax')))
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    return model
tests.test_encdec_model(encdec_model)


# OPTIONAL: Train and Print prediction(s)
tmp_x = pad(preproc_english_sentences)
tmp_x = tmp_x.reshape((-1, preproc_english_sentences.shape[1], 1))

encodeco_model = encdec_model(
    tmp_x.shape,
    preproc_french_sentences.shape[1],
    len(english_tokenizer.word_index)+1,
    len(french_tokenizer.word_index)+1)

encodeco_model.fit(tmp_x, preproc_french_sentences, batch_size=1024, epochs=20, validation_split=0.2)

print(logits_to_text(encodeco_model.predict(tmp_x[:1])[0], french_tokenizer))






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 110288 samples, validate on 27573 samples
Epoch 1/20





 43008/110288 [==========>...................] - ETA: 42s - loss: 3.6586 - acc: 0.4097

KeyboardInterrupt: 

### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [36]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    model = Sequential()
    model.add(Embedding(input_dim=english_vocab_size,output_dim=256,input_length=input_shape[1]))
    model.add(Bidirectional(GRU(256,return_sequences=False)))
    model.add(RepeatVector(output_sequence_length))
    model.add(Bidirectional(GRU(256,return_sequences=True)))
    model.add(TimeDistributed(Dense(french_vocab_size,activation='softmax')))
    learning_rate = 0.005
    
    model.compile(loss = sparse_categorical_crossentropy, 
                 optimizer = Adam(learning_rate), 
                 metrics = ['accuracy'])
    
    return model
tests.test_model_final(model_final)


print('Final Model Loaded')
# TODO: Train the final model

Final Model Loaded


In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(english_vocab_size, 64),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(256)),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(french_vocab_size.shape[1])
])

NameError: name 'tf' is not defined

## Prediction (IMPLEMENTATION)

In [ ]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    tmp_X = pad(preproc_english_sentences)
    model = model_final(tmp_X.shape,
                        preproc_french_sentences.shape[1],
                        len(english_tokenizer.word_index)+1,
                        len(french_tokenizer.word_index)+1)
    
    model.fit(tmp_X, preproc_french_sentences, batch_size = 1024, epochs = 17, validation_split = 0.2)
 
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Train on 266768 samples, validate on 66692 samples
Epoch 1/17


## Submission
When you're ready to submit, complete the following steps:
1. Review the [rubric](https://review.udacity.com/#!/rubrics/1004/view) to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [18]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 345566 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?